In [1]:
import argparse
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sched_heuristic as heu
import pickle
from scipy.stats import kendalltau as tau, spearmanr as rho
from collections import defaultdict
from sched_solver import random_shuffle
import util as util_
from concurrent.futures import ProcessPoolExecutor

In [2]:
num_proc = 2
use_deadline = True
num_tasks = 8

In [3]:
!ls | grep torchmodel

In [4]:
ret = []
beg = "10"
end = "190"
start=False
for x in [str(x) for x in range(10, num_proc * 100, 10)]:
    if beg == x:
        start = True
    if start:
        if len(x) == 2:
            ret.append('0.' + x)
        else:
            ret.append(x[:len(x) - 2] + '.' + x[len(x) - 2:])
    if end == x:
        start = False

In [5]:
util_range = ret

In [6]:
util_range

['0.10',
 '0.20',
 '0.30',
 '0.40',
 '0.50',
 '0.60',
 '0.70',
 '0.80',
 '0.90',
 '1.00',
 '1.10',
 '1.20',
 '1.30',
 '1.40',
 '1.50',
 '1.60',
 '1.70',
 '1.80',
 '1.90']

In [7]:
model = torch.load("models/p2-t8-d1-l[0.10, 0.50].torchmodel")

In [8]:
executor = executor = ProcessPoolExecutor(max_workers=4)

In [9]:
def wrap(input):
    res_map = defaultdict(lambda: list())
    idx, num_tasks, num_procs, util, model  = input 
    with open("eval/%d-%d/%s" % (num_proc, num_tasks, util), 'rb') as f:
        train_dataset = pickle.load(f)
    i = 0
    for x, y in train_dataset:
        if i == 10000:
            break
        i += 1
        if i % 10 == 0:
            print("idx",idx, "iter", i)
        priorities = random_shuffle(model, y, 256)
        true = False
        for priority in priorities:
            ret = heu.test_RTA_LC(y, num_proc, priority, True)
            if ret:
                true = True
                break
        res_map[util].append(true)
    return dict(res_map)

In [10]:
utils = ["0.10", "0.20", "0.30", "0.40"]
ret = []
for x in executor.map(wrap, [(i, num_tasks, num_proc, util, model) for i, util in enumerate(utils)]):
    ret.append(x)
a = {}
for x in ret:
    a.update(x)
a

idx 2 iter 10
idx 1 iter 10
idx 0 iter 10
idx 3 iter 10
idx 2 iter 20
idx 3 iter 20
idx 0 iter 20
idx 1 iter 20
idx 3 iter 30
idx 2 iter 30
idx 0 iter 30
idx 1 iter 30
idx 2 iter 40
idx 0 iter 40
idx 3 iter 40
idx 1 iter 40
idx 0 iter 50
idx 2 iter 50
idx 3 iter 50
idx 1 iter 50
idx 0 iter 60
idx 2 iter 60
idx 3 iter 60
idx 1 iter 60
idx 0 iter 70
idx 3 iter 70
idx 2 iter 70
idx 1 iter 70
idx 0 iter 80
idx 3 iter 80
idx 2 iter 80
idx 1 iter 80
idx 0 iter 90
idx 3 iter 90
idx 2 iter 90
idx 1 iter 90
idx 0 iter 100
idx 3 iter 100
idx 2 iter 100
idx 1 iter 100
idx 0 iter 110
idx 3 iter 110
idx 2 iter 110
idx 1 iter 110
idx 0 iter 120
idx 3 iter 120
idx 2 iter 120
idx 1 iter 120
idx 0 iter 130
idx 3 iter 130
idx 2 iter 130
idx 1 iter 130
idx 0 iter 140
idx 3 iter 140
idx 2 iter 140
idx 1 iter 140
idx 0 iter 150
idx 3 iter 150
idx 2 iter 150
idx 1 iter 150
idx 0 iter 160
idx 3 iter 160
idx 2 iter 160
idx 1 iter 160
idx 0 iter 170
idx 3 iter 170
idx 2 iter 170
idx 1 iter 170
idx 0 iter 180
i

idx 3 iter 1370
idx 0 iter 1400
idx 1 iter 1390
idx 2 iter 1390
idx 3 iter 1380
idx 0 iter 1410
idx 1 iter 1400
idx 2 iter 1400
idx 3 iter 1390
idx 0 iter 1420
idx 1 iter 1410
idx 2 iter 1410
idx 3 iter 1400
idx 0 iter 1430
idx 1 iter 1420
idx 2 iter 1420
idx 3 iter 1410
idx 0 iter 1440
idx 1 iter 1430
idx 3 iter 1420
idx 2 iter 1430
idx 0 iter 1450
idx 1 iter 1440
idx 2 iter 1440
idx 3 iter 1430
idx 0 iter 1460
idx 1 iter 1450
idx 3 iter 1440
idx 2 iter 1450
idx 0 iter 1470
idx 1 iter 1460
idx 3 iter 1450
idx 2 iter 1460
idx 0 iter 1480
idx 1 iter 1470
idx 3 iter 1460
idx 2 iter 1470
idx 0 iter 1490
idx 1 iter 1480
idx 3 iter 1470
idx 2 iter 1480
idx 0 iter 1500
idx 1 iter 1490
idx 3 iter 1480
idx 2 iter 1490
idx 0 iter 1510
idx 1 iter 1500
idx 3 iter 1490
idx 2 iter 1500
idx 0 iter 1520
idx 1 iter 1510
idx 3 iter 1500
idx 2 iter 1510
idx 0 iter 1530
idx 1 iter 1520
idx 3 iter 1510
idx 2 iter 1520
idx 0 iter 1540
idx 1 iter 1530
idx 3 iter 1520
idx 2 iter 1530
idx 0 iter 1550
idx 1 it

idx 3 iter 2700
idx 2 iter 2730
idx 0 iter 2760
idx 1 iter 2740
idx 3 iter 2710
idx 2 iter 2740
idx 1 iter 2750
idx 0 iter 2770
idx 3 iter 2720
idx 2 iter 2750
idx 1 iter 2760
idx 0 iter 2780
idx 3 iter 2730
idx 2 iter 2760
idx 1 iter 2770
idx 0 iter 2790
idx 3 iter 2740
idx 2 iter 2770
idx 1 iter 2780
idx 0 iter 2800
idx 3 iter 2750
idx 2 iter 2780
idx 0 iter 2810
idx 1 iter 2790
idx 3 iter 2760
idx 2 iter 2790
idx 0 iter 2820
idx 1 iter 2800
idx 3 iter 2770
idx 2 iter 2800
idx 0 iter 2830
idx 1 iter 2810
idx 3 iter 2780
idx 2 iter 2810
idx 0 iter 2840
idx 1 iter 2820
idx 3 iter 2790
idx 2 iter 2820
idx 1 iter 2830
idx 0 iter 2850
idx 3 iter 2800
idx 2 iter 2830
idx 1 iter 2840
idx 0 iter 2860
idx 3 iter 2810
idx 2 iter 2840
idx 1 iter 2850
idx 0 iter 2870
idx 3 iter 2820
idx 2 iter 2850
idx 1 iter 2860
idx 0 iter 2880
idx 3 iter 2830
idx 2 iter 2860
idx 1 iter 2870
idx 0 iter 2890
idx 3 iter 2840
idx 2 iter 2870
idx 1 iter 2880
idx 0 iter 2900
idx 3 iter 2850
idx 2 iter 2880
idx 1 it

idx 0 iter 4110
idx 2 iter 4080
idx 1 iter 4100
idx 3 iter 4050
idx 0 iter 4120
idx 2 iter 4090
idx 1 iter 4110
idx 3 iter 4060
idx 0 iter 4130
idx 2 iter 4100
idx 1 iter 4120
idx 3 iter 4070
idx 0 iter 4140
idx 2 iter 4110
idx 1 iter 4130
idx 3 iter 4080
idx 0 iter 4150
idx 2 iter 4120
idx 1 iter 4140
idx 3 iter 4090
idx 0 iter 4160
idx 2 iter 4130
idx 1 iter 4150
idx 3 iter 4100
idx 0 iter 4170
idx 2 iter 4140
idx 1 iter 4160
idx 3 iter 4110
idx 0 iter 4180
idx 2 iter 4150
idx 1 iter 4170
idx 3 iter 4120
idx 0 iter 4190
idx 2 iter 4160
idx 1 iter 4180
idx 3 iter 4130
idx 0 iter 4200
idx 2 iter 4170
idx 1 iter 4190
idx 3 iter 4140
idx 0 iter 4210
idx 2 iter 4180
idx 1 iter 4200
idx 3 iter 4150
idx 0 iter 4220
idx 2 iter 4190
idx 1 iter 4210
idx 3 iter 4160
idx 0 iter 4230
idx 2 iter 4200
idx 1 iter 4220
idx 0 iter 4240
idx 3 iter 4170
idx 1 iter 4230
idx 2 iter 4210
idx 0 iter 4250
idx 3 iter 4180
idx 1 iter 4240
idx 2 iter 4220
idx 0 iter 4260
idx 3 iter 4190
idx 2 iter 4230
idx 1 it

idx 3 iter 5320
idx 0 iter 5410
idx 2 iter 5360
idx 1 iter 5380
idx 3 iter 5330
idx 0 iter 5420
idx 2 iter 5370
idx 1 iter 5390
idx 3 iter 5340
idx 0 iter 5430
idx 2 iter 5380
idx 1 iter 5400
idx 3 iter 5350
idx 0 iter 5440
idx 1 iter 5410
idx 2 iter 5390
idx 3 iter 5360
idx 0 iter 5450
idx 1 iter 5420
idx 2 iter 5400
idx 3 iter 5370
idx 0 iter 5460
idx 1 iter 5430
idx 2 iter 5410
idx 3 iter 5380
idx 0 iter 5470
idx 1 iter 5440
idx 2 iter 5420
idx 0 iter 5480
idx 3 iter 5390
idx 1 iter 5450
idx 2 iter 5430
idx 0 iter 5490
idx 3 iter 5400
idx 1 iter 5460
idx 2 iter 5440
idx 0 iter 5500
idx 3 iter 5410
idx 1 iter 5470
idx 2 iter 5450
idx 0 iter 5510
idx 3 iter 5420
idx 1 iter 5480
idx 2 iter 5460
idx 0 iter 5520
idx 3 iter 5430
idx 1 iter 5490
idx 2 iter 5470
idx 0 iter 5530
idx 3 iter 5440
idx 1 iter 5500
idx 2 iter 5480
idx 0 iter 5540
idx 3 iter 5450
idx 1 iter 5510
idx 0 iter 5550
idx 2 iter 5490
idx 3 iter 5460
idx 1 iter 5520
idx 0 iter 5560
idx 2 iter 5500
idx 3 iter 5470
idx 1 it

idx 1 iter 6660
idx 0 iter 6700
idx 2 iter 6640
idx 3 iter 6600
idx 1 iter 6670
idx 2 iter 6650
idx 0 iter 6710
idx 3 iter 6610
idx 1 iter 6680
idx 2 iter 6660
idx 0 iter 6720
idx 3 iter 6620
idx 1 iter 6690
idx 2 iter 6670
idx 0 iter 6730
idx 3 iter 6630
idx 1 iter 6700
idx 0 iter 6740
idx 2 iter 6680
idx 3 iter 6640
idx 0 iter 6750
idx 1 iter 6710
idx 2 iter 6690
idx 3 iter 6650
idx 0 iter 6760
idx 1 iter 6720
idx 2 iter 6700
idx 3 iter 6660
idx 1 iter 6730
idx 0 iter 6770
idx 2 iter 6710
idx 3 iter 6670
idx 1 iter 6740
idx 0 iter 6780
idx 2 iter 6720
idx 3 iter 6680
idx 1 iter 6750
idx 0 iter 6790
idx 2 iter 6730
idx 3 iter 6690
idx 1 iter 6760
idx 0 iter 6800
idx 2 iter 6740
idx 3 iter 6700
idx 0 iter 6810
idx 1 iter 6770
idx 2 iter 6750
idx 3 iter 6710
idx 0 iter 6820
idx 1 iter 6780
idx 2 iter 6760
idx 3 iter 6720
idx 0 iter 6830
idx 1 iter 6790
idx 2 iter 6770
idx 3 iter 6730
idx 0 iter 6840
idx 1 iter 6800
idx 2 iter 6780
idx 3 iter 6740
idx 0 iter 6850
idx 1 iter 6810
idx 2 it

idx 2 iter 7910
idx 1 iter 7940
idx 0 iter 8000
idx 2 iter 7920
idx 3 iter 7880
idx 1 iter 7950
idx 0 iter 8010
idx 2 iter 7930
idx 1 iter 7960
idx 3 iter 7890
idx 0 iter 8020
idx 2 iter 7940
idx 1 iter 7970
idx 3 iter 7900
idx 0 iter 8030
idx 2 iter 7950
idx 3 iter 7910
idx 1 iter 7980
idx 0 iter 8040
idx 2 iter 7960
idx 3 iter 7920
idx 1 iter 7990
idx 0 iter 8050
idx 2 iter 7970
idx 3 iter 7930
idx 1 iter 8000
idx 0 iter 8060
idx 2 iter 7980
idx 1 iter 8010
idx 3 iter 7940
idx 0 iter 8070
idx 2 iter 7990
idx 1 iter 8020
idx 3 iter 7950
idx 0 iter 8080
idx 2 iter 8000
idx 1 iter 8030
idx 3 iter 7960
idx 0 iter 8090
idx 2 iter 8010
idx 1 iter 8040
idx 3 iter 7970
idx 0 iter 8100
idx 2 iter 8020
idx 1 iter 8050
idx 0 iter 8110
idx 3 iter 7980
idx 2 iter 8030
idx 0 iter 8120
idx 1 iter 8060
idx 3 iter 7990
idx 2 iter 8040
idx 0 iter 8130
idx 1 iter 8070
idx 3 iter 8000
idx 2 iter 8050
idx 0 iter 8140
idx 1 iter 8080
idx 3 iter 8010
idx 2 iter 8060
idx 1 iter 8090
idx 3 iter 8020
idx 0 it

idx 0 iter 9290
idx 3 iter 9140
idx 1 iter 9230
idx 2 iter 9200
idx 0 iter 9300
idx 3 iter 9150
idx 1 iter 9240
idx 2 iter 9210
idx 0 iter 9310
idx 3 iter 9160
idx 1 iter 9250
idx 2 iter 9220
idx 0 iter 9320
idx 3 iter 9170
idx 1 iter 9260
idx 2 iter 9230
idx 0 iter 9330
idx 3 iter 9180
idx 1 iter 9270
idx 2 iter 9240
idx 0 iter 9340
idx 3 iter 9190
idx 1 iter 9280
idx 2 iter 9250
idx 0 iter 9350
idx 3 iter 9200
idx 1 iter 9290
idx 2 iter 9260
idx 0 iter 9360
idx 3 iter 9210
idx 1 iter 9300
idx 2 iter 9270
idx 0 iter 9370
idx 3 iter 9220
idx 1 iter 9310
idx 2 iter 9280
idx 0 iter 9380
idx 3 iter 9230
idx 1 iter 9320
idx 2 iter 9290
idx 0 iter 9390
idx 3 iter 9240
idx 1 iter 9330
idx 2 iter 9300
idx 0 iter 9400
idx 3 iter 9250
idx 1 iter 9340
idx 2 iter 9310
idx 0 iter 9410
idx 3 iter 9260
idx 1 iter 9350
idx 2 iter 9320
idx 0 iter 9420
idx 3 iter 9270
idx 1 iter 9360
idx 0 iter 9430
idx 2 iter 9330
idx 3 iter 9280
idx 1 iter 9370
idx 0 iter 9440
idx 2 iter 9340
idx 3 iter 9290
idx 1 it

{'0.10': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,


In [12]:
with open("2-8-dl-0.10-0.40.pkl", 'wb') as f:
    pickle.dump(a, f)

In [15]:
for k ,i in a.items():
    print(np.sum(i))

9989
9982
9978
9947
